<h1>Allocation Service - Notebook 1</h1>
This notebook demonstrates a simple use case of the Allocation Service. We will create two orders for the same instrument for separate portfolios, along with the supporting entities. We will see how the Allocation Service handles two executions.

<h2> Imports </h2>
Import the required packages for this example.

In [1]:
import lusid
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)
from helpers import AllocationServiceHelpers as helpers

import pandas as pd
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.11171.0


<h2> Setup Constants </h2>
We set up everything in the same scope for simplicity.

In [2]:
scope = "AllocationService_Example1"

<h2> Upsert Instrument </h2>
We use a simple AMZN equity for this example.

In [3]:
instr_df = pd.read_csv("data/1/instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,ticker
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,AMZN


In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


<h2> Upsert Portfolios </h2>
We have two different portfolios, which will have their own respective orders.

In [5]:
pf_df = pd.read_csv("data/1/portfolios.csv")
display(pf_df)

,portfolio_name,portfolio_code,currency,created_date
0,Allocation Service - Example 1 - Portfolio A,PORT001,USD,2023-04-10T12:00:00+00:00
1,Allocation Service - Example 2 - Portfolio B,PORT002,USD,2023-04-10T12:00:00+00:00


In [6]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "currency",
    }
}
result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional={},
        file_type="portfolios",
    )

succ, errors = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "errors": len(errors)}])

,success,errors
0,2,0


<h2> Upsert Orders </h2>
Here we have two orders for AMZN stock, both for ten units each with Order 1 being placed 5 minutes before Order 2.

In [21]:
ord_df = pd.read_csv("data/1/orders.csv")
display(ord_df)

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,type,state,date
0,PORT001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,1550,USD,ORD001,buy,limit,new,2023-04-13T11:10:00+00:00
1,PORT002,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,10,1550,USD,ORD002,buy,limit,new,2023-04-13T11:15:00+00:00


In [22]:
set_request = helpers.create_order_set_request_from_df(ord_df, scope)
response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
    order_set_request=set_request)

display(lusid_response_to_data_frame(response))

,properties,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number,instrument_identifiers.Instrument/default/Figi,quantity,side,portfolio_id.scope,portfolio_id.code,id.scope,id.code,instrument_scope,lusid_instrument_id,state,type,date,price.amount,price.currency
0,{},0001-01-01 00:00:00+00:00,2023-04-21 15:11:21.760756+00:00,2023-04-13 13:48:11.613070+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:11:21.760756+00:00,00ucmrzqsbhjXJHmz2p7,14,BBG000BVPXP1,10.0,buy,AllocationService_Example1,PORT002,AllocationService_Example1,ORD002,default,LUID_0000MIOX,new,limit,2023-04-13 11:15:00+00:00,10.0,USD
1,{},0001-01-01 00:00:00+00:00,2023-04-21 15:11:21.760756+00:00,2023-04-13 13:48:11.613070+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:11:21.760756+00:00,00ucmrzqsbhjXJHmz2p7,14,BBG000BVPXP1,10.0,buy,AllocationService_Example1,PORT001,AllocationService_Example1,ORD001,default,LUID_0000MIOX,new,limit,2023-04-13 11:10:00+00:00,10.0,USD


<h2> Upsert Block </h2>
The Block contains both of our orders.

In [9]:
block_df = pd.read_csv("data/1/blocks.csv")
display(block_df)

,block_id,order_ids,instrument_name,client_internal,isin,figi,quantity,side,type,time_in_force,created_date
0,BLOCK001,"ORD001,ORD002",Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,20,buy,limit,GTC,2023-04-13T11:15:00+00:00


In [10]:
set_request = helpers.create_block_set_request_from_df(block_df, scope)
response = api_factory.build(lusid.api.BlocksApi).upsert_blocks(
    block_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,order_ids.0.scope,order_ids.0.code,order_ids.1.scope,order_ids.1.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,side,type,time_in_force,created_date,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example1,BLOCK001,AllocationService_Example1,ORD001,AllocationService_Example1,ORD002,{},BBG000BVPXP1,LUID_0000MIOX,20.0,buy,limit,GTC,2023-04-13 11:15:00+00:00,0001-01-01 00:00:00+00:00,2023-04-21 15:02:16.095784+00:00,2023-04-13 13:57:28.785428+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:02:16.095784+00:00,00ucmrzqsbhjXJHmz2p7,11


<h2> Upsert Placement </h2>
The Placement references our Block.

In [11]:
placement_df = pd.read_csv("data/1/placements.csv")
display(placement_df)

,placement_id,block_id,instrument_name,client_internal,isin,figi,quantity,state,side,type,time_in_force,created_date
0,PLC001,BLOCK001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,20,some_state,buy,limit,GTC,2023-04-13T11:15:00+00:00


In [12]:
set_request = helpers.create_placement_set_request_from_df(placement_df, scope)
response = api_factory.build(lusid.api.PlacementsApi).upsert_placements(
    placement_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,block_ids.0.scope,block_ids.0.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,state,side,time_in_force,type,created_date,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example1,PLC001,AllocationService_Example1,BLOCK001,{},BBG000BVPXP1,LUID_0000MIOX,20.0,some_state,buy,GTC,limit,2023-04-13 11:15:00+00:00,0001-01-01 00:00:00+00:00,2023-04-21 15:02:16.317825+00:00,2023-04-13 14:08:39.961121+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:02:16.317825+00:00,00ucmrzqsbhjXJHmz2p7,9


<h2> Upsert Execution 1 </h2>
Our first execution has a quantitiy of 5 and was executed at a price of 1550.

In [13]:
execution_df = pd.read_csv("data/1/executions.csv").head(1)
display(execution_df)

,execution_id,placement_id,instrument_name,client_internal,isin,figi,quantity,state,side,type,price,currency,settlement_currency,settlement_currency_fx_rate,counterparty,created_date
0,EXEC001,PLC001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,5,some_state,buy,limit,1550,USD,USD,1.0,some_counterparty,2023-04-13T11:15:00+00:00


In [14]:
set_request = helpers.create_execution_set_request_from_df(execution_df, scope)
response = api_factory.build(lusid.api.ExecutionsApi).upsert_executions(
    execution_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,placement_id.scope,placement_id.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,state,side,type,created_date,price.amount,price.currency,settlement_currency,settlement_currency_fx_rate,counterparty,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example1,EXEC001,AllocationService_Example1,PLC001,{},BBG000BVPXP1,LUID_0000MIOX,5.0,some_state,buy,limit,2023-04-13 11:15:00+00:00,1550.0,USD,USD,1.0,some_counterparty,0001-01-01 00:00:00+00:00,2023-04-21 15:02:16.533945+00:00,2023-04-13 14:16:49.278810+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:02:16.533945+00:00,00ucmrzqsbhjXJHmz2p7,7


<h2> Run the Allocation Service </h2>
You can see that the placement ID for our placement is in the values section, this means that the Allocation Service has successfully allocated our executions.

In [23]:
result = api_factory.build(lusid.api.AllocationServiceApi).run_allocation_service([lusid.models.ResourceId(scope,"PLC001")])
display(result)

{'failed': {},
 'values': [{'code': 'PLC001', 'scope': 'AllocationService_Example1'}]}

<h2> Retrieve our Allocations </h2>
You can see that we get two allocations, one for each order. They both have a price of 1550 to match the execution. The allocation for Order 1 has a quantity of 3 as Order 1 was placed earlier hence it gets allocated the orphan share.

In [24]:
allocations = api_factory.build(lusid.api.AllocationsApi).list_allocations(filter=f"allocatedOrderId.Scope eq \'{scope}\' and allocatedOrderId.Code in (\'ORD001', \'ORD002\')")
display(lusid_response_to_data_frame(allocations))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/LusidInstrumentId,version.effective_from,version.as_at_date,properties,instrument_scope,lusid_instrument_id,placement_ids.0.scope,placement_ids.0.code,state,side,type,date,price.amount,price.currency,settlement_currency,execution_ids.0.scope,execution_ids.0.code
0,AllocationService_Example1,4172acef-4a45-48f5-8965-0f7f5ce8eeb6,AllocationService_Example1,ORD001,AllocationService_Example1,PORT001,3.0,LUID_0000MIOX,0001-01-01 00:00:00+00:00,2023-04-21 15:11:43.265158+00:00,{},default,LUID_0000MIOX,AllocationService_Example1,PLC001,PartiallyAllocated,buy,limit,2023-04-21 14:53:34.253167+00:00,1550.0,USD,USD,AllocationService_Example1,EXEC001
1,AllocationService_Example1,38cbda2a-478b-40e0-a793-7df537c2dea9,AllocationService_Example1,ORD002,AllocationService_Example1,PORT002,2.0,LUID_0000MIOX,0001-01-01 00:00:00+00:00,2023-04-21 15:11:43.265158+00:00,{},default,LUID_0000MIOX,AllocationService_Example1,PLC001,PartiallyAllocated,buy,limit,2023-04-21 14:53:34.250409+00:00,1550.0,USD,USD,AllocationService_Example1,EXEC001


<h2> Upsert Execution 2 </h2>
Our second execution again has a quantity of 5, but a price of 1560.

In [17]:
execution_df = pd.read_csv("data/1/executions.csv")
display(execution_df)

,execution_id,placement_id,instrument_name,client_internal,isin,figi,quantity,state,side,type,price,currency,settlement_currency,settlement_currency_fx_rate,counterparty,created_date
0,EXEC001,PLC001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,5,some_state,buy,limit,1550,USD,USD,1.0,some_counterparty,2023-04-13T11:15:00+00:00
1,EXEC002,PLC001,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,5,some_state,buy,limit,1560,USD,USD,1.0,some_counterparty,2023-04-13T11:20:00+00:00


In [18]:
set_request = helpers.create_execution_set_request_from_df(execution_df, scope)
response = api_factory.build(lusid.api.ExecutionsApi).upsert_executions(
    execution_set_request=set_request)

display(lusid_response_to_data_frame(response))

,id.scope,id.code,placement_id.scope,placement_id.code,properties,instrument_identifiers.Instrument/default/Figi,lusid_instrument_id,quantity,state,side,type,created_date,price.amount,price.currency,settlement_currency,settlement_currency_fx_rate,counterparty,version.effective_from,version.as_at_date,version.as_at_created,version.user_id_created,version.as_at_modified,version.user_id_modified,version.as_at_version_number
0,AllocationService_Example1,EXEC001,AllocationService_Example1,PLC001,{},BBG000BVPXP1,LUID_0000MIOX,5.0,some_state,buy,limit,2023-04-13 11:15:00+00:00,1550.0,USD,USD,1.0,some_counterparty,0001-01-01 00:00:00+00:00,2023-04-21 15:02:18.169401+00:00,2023-04-13 14:16:49.278810+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:02:18.169401+00:00,00ucmrzqsbhjXJHmz2p7,8
1,AllocationService_Example1,EXEC002,AllocationService_Example1,PLC001,{},BBG000BVPXP1,LUID_0000MIOX,5.0,some_state,buy,limit,2023-04-13 11:20:00+00:00,1560.0,USD,USD,1.0,some_counterparty,0001-01-01 00:00:00+00:00,2023-04-21 15:02:18.169401+00:00,2023-04-21 15:02:18.169401+00:00,00ucmrzqsbhjXJHmz2p7,2023-04-21 15:02:18.169401+00:00,00ucmrzqsbhjXJHmz2p7,1


<h2> Run the Allocation Service again </h2>
You can see that the placement ID for our placement is in the values section, this means that the Allocation Service has successfully allocated our executions.

In [19]:
result = api_factory.build(lusid.api.AllocationServiceApi).run_allocation_service([lusid.models.ResourceId(scope,"PLC001")])
display(result)

{'failed': {},
 'values': [{'code': 'PLC001', 'scope': 'AllocationService_Example1'}]}

<h2> Retrieve our Allocations again </h2>
Here you can see that our price has been averaged to 1555 and that the quantity for both Allocations is now 5. This is due to them being allocated as one grouped execution of 10 instead of 2 lots of 5.

In [20]:
allocations = api_factory.build(lusid.api.AllocationsApi).list_allocations(filter=f"allocatedOrderId.Scope eq \'{scope}\' and allocatedOrderId.Code in (\'ORD001', \'ORD002\')")
display(lusid_response_to_data_frame(allocations))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/LusidInstrumentId,version.effective_from,version.as_at_date,properties,instrument_scope,lusid_instrument_id,placement_ids.0.scope,placement_ids.0.code,state,side,type,date,price.amount,price.currency,settlement_currency,execution_ids.0.scope,execution_ids.0.code,execution_ids.1.scope,execution_ids.1.code
0,AllocationService_Example1,4172acef-4a45-48f5-8965-0f7f5ce8eeb6,AllocationService_Example1,ORD001,AllocationService_Example1,PORT001,5.0,LUID_0000MIOX,0001-01-01 00:00:00+00:00,2023-04-21 15:02:18.590382+00:00,{},default,LUID_0000MIOX,AllocationService_Example1,PLC001,PartiallyAllocated,buy,limit,2023-04-21 14:53:34.253167+00:00,1555.0,USD,USD,AllocationService_Example1,EXEC001,AllocationService_Example1,EXEC002
1,AllocationService_Example1,38cbda2a-478b-40e0-a793-7df537c2dea9,AllocationService_Example1,ORD002,AllocationService_Example1,PORT002,5.0,LUID_0000MIOX,0001-01-01 00:00:00+00:00,2023-04-21 15:02:18.590382+00:00,{},default,LUID_0000MIOX,AllocationService_Example1,PLC001,PartiallyAllocated,buy,limit,2023-04-21 14:53:34.250409+00:00,1555.0,USD,USD,AllocationService_Example1,EXEC001,AllocationService_Example1,EXEC002
